<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Workshop SWDB 2023, Day 5 </h1> 
<h3 align="center"> August 24, 2023</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> The Questions </h2>
    
<p> The first week of the course is devoted to question focused workshops and addresing those questions with datasets and computational methods that we will present.  Each day will be devoted to its own question.  
    
<p> For reference, the questions (as they are currently formulated) are
<ul>
<li> Day 1:  How do single neurons encode information about the external world?  
<li> Day 2:  How do populations of neurons encode information about the external world? 
<li> Day 3:  How do cell type and circuit architecture contribute to physiological function?
<li> Day 4:  What is a behavioral state and how can we define it? 
<b><li> Day 5:  How does network structure lead to network dynamics?</b>
</ul>

### Importing of library dependencies

In [4]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec # Useful tool to arrange multiple plots in one figure (https://matplotlib.org/stable/api/_as_gen/matplotlib.gridspec.GridSpec.html)
%matplotlib inline

import torch # to determine if we can do away with it
import seaborn as sns


## data loading

In [2]:
from allensdk.brain_observatory.behavior.behavior_project_cache.\
    behavior_neuropixels_project_cache \
    import VisualBehaviorNeuropixelsProjectCache

In [6]:
# where the data will be stored
output_dir = "/src/swdb_2023/Day5/data"

cache = VisualBehaviorNeuropixelsProjectCache.from_s3_cache(
            cache_dir=Path(output_dir))

ecephys_sessions.csv: 100%|██████████| 64.7k/64.7k [00:00<00:00, 486kMB/s] 
behavior_sessions.csv: 100%|██████████| 562k/562k [00:00<00:00, 5.91MMB/s]
units.csv: 100%|██████████| 132M/132M [00:05<00:00, 25.9MMB/s]   
probes.csv: 100%|██████████| 130k/130k [00:00<00:00, 1.00MMB/s] 
channels.csv: 100%|██████████| 27.9M/27.9M [00:01<00:00, 20.7MMB/s]  


In [7]:
# get the metadata tables
units_table = cache.get_unit_table()
channels_table = cache.get_channel_table()
probes_table = cache.get_probe_table()
behavior_sessions_table = cache.get_behavior_session_table()
ecephys_sessions_table = cache.get_ecephys_session_table()

Filtering the sessions table for a session of interest

In [8]:
sst_novel_sessions = ecephys_sessions_table.loc[(ecephys_sessions_table['genotype'].str.contains('Sst')) & 
                                            (ecephys_sessions_table['experience_level']=='Novel')]
sst_novel_sessions.head()

,behavior_session_id,date_of_acquisition,equipment_name,session_type,mouse_id,genotype,sex,project_code,age_in_days,unit_count,...,channel_count,structure_acronyms,image_set,prior_exposures_to_image_set,session_number,experience_level,prior_exposures_to_omissions,file_id,abnormal_histology,abnormal_activity
ecephys_session_id,,,,,,,,,,,,,,,,,,,,,
1048189115,1048221709,2020-09-03 14:16:57.913000+00:00,NP.1,EPHYS_1_images_H_3uL_reward,509808,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,M,NeuropixelVisualBehavior,264,1925.0,...,2304.0,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0.0,2,Novel,1,879,NaN,NaN
1048196054,1048222325,2020-09-03 14:25:07.290000+00:00,NP.0,EPHYS_1_images_H_3uL_reward,524925,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,F,NeuropixelVisualBehavior,166,2288.0,...,2304.0,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0.0,2,Novel,1,880,NaN,NaN
1053941483,1053960987,2020-10-01 17:03:58.362000+00:00,NP.1,EPHYS_1_images_H_3uL_reward,527749,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,M,NeuropixelVisualBehavior,180,1543.0,...,2304.0,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0.0,2,Novel,1,891,NaN,NaN
1064644573,1064666428,2020-11-19 15:18:01.372000+00:00,NP.1,EPHYS_1_images_H_3uL_reward,544456,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,M,NeuropixelVisualBehavior,120,2254.0,...,2304.0,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0.0,2,Novel,1,909,NaN,NaN
1065905010,1065929713,2020-11-24 14:21:48.847000+00:00,NP.0,EPHYS_1_images_H_3uL_reward,544358,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,F,NeuropixelVisualBehavior,126,1998.0,...,2304.0,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0.0,2,Novel,1,912,NaN,NaN


# Workshop 1: Visualizing and modeling the dynamic variability of a single neuron

## Variability in neural population responses
We will first take a look at exactly how variable neural population responses are by looking at each neuron's responses to visual stimuli.

Things ToDo:
1. Load visual behavior dataset
2. Extract responses of example neurons to identical visual stimuli and plot the histogram in responses
3. Repeat 2, but this time aligned relative to when the stimulus frame just changed
4. Repeat 3, but this time for unfamliar (first session when introduce) vs familiar (in later sessions)
5. If possible, condition the visualization on behavioral states from Day 4

## Predicting the neural responses

Here we will build models to predict the neural responses.

# Workshop 2: Visualizing and modeling the dynamic variability of a neural population

## Visual behavior dataset
1. Load visual behavior dataset
2. Predict neural population responses from visual stimuli (NOTE: this is no different then just running individual neuron prediction independently UNLESS noise model differs -> Exercise point)
3. Improve on neural population response prediction by including linear interaction with previous time-step neural population responses.
4. Assess the inferred interaction between neurons (based on weights). Can anything be said about the inferred functional connectivity?

## BONUS: Relating infererd neural interaction in models to 

## MIcRONS dataset

1. Perform neural population response fit and idetnify "functional" connectivity
2. Relate the functional connectivity to known connectivity

1. Load MICrONS dataset
2. Predict neural 